In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import pandas as pd

In [2]:
url = 'https://www.yelp.co.uk'
driver = webdriver.Safari()
driver.get(url)
time.sleep(3)


#### Get rid of cookies warning

In [3]:
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'onetrust-accept-btn-handler'))).click()



##### Anyone can change these two elements to persolize it

In [4]:
search_box = driver.find_element(By.NAME, value='find_desc')
search_box.click()
search_box.send_keys('Restaurants')
location = driver.find_element(By.ID, value='search_location')
location.click()
location.clear()
location.send_keys('Liverpool')

In [5]:

search_button = driver.find_element(By.XPATH, value='//*[@id="header_find_form"]/div/div[2]/div/div/button')
search_button.click()
time.sleep(3)

In [6]:
pages = driver.find_element(By.XPATH, value='//div[@class=" border-color--default__09f24__NPAKY text-align--center__09f24__fYBGO"]/span').text.split(' ')[-1]
pages

'24'

In [8]:
name_list = []
stars_list = []
num_of_reviews_list = []
address_list = []
while True:
    try:
        for i in range(int(pages)):
            name = driver.find_elements(By.XPATH, value="//div[@class=' businessName__09f24__EYSZE display--inline-block__09f24__fEDiJ border-color--default__09f24__NPAKY']/div/h3/span/a")
            num_of_reviews = driver.find_elements(By.XPATH, value='//div[@class=" display--inline-block__09f24__fEDiJ border-color--default__09f24__NPAKY"]/span[@class=" css-chan6m"]')
            stars = driver.find_elements(By.XPATH, value='//div[@class=" border-color--default__09f24__NPAKY"]/div/div/span/div')
            address = driver.find_elements(By.XPATH, value='//p[@class="css-dzq7l1"]/span[@class=" css-chan6m"]')
            all_lists_length = [
                len(name),
                len(stars),
                len(num_of_reviews),
                len(address)
            ]

            if len(set(all_lists_length)) == 1:
                for i in range(all_lists_length[0]):
                    name_list.append(name[i].text)
                    stars_list.append(float(stars[i].get_attribute('aria-label').split(' ')[0]))
                    num_of_reviews_list.append(int(num_of_reviews[i].text))
                    address_list.append(address[i].text)
            else:
                print('Sorry! Some values are missing.')
            next_page = driver.find_element(By.XPATH, value='//li[@class=" border-color--default__09f24__NPAKY"]/div/div/div/div/span/a[@class="next-link navigation-button__09f24__m9qRz css-144i0wq"]')
            next_page.click()
            time.sleep(3)
    except:
        break

In [9]:
all_lists_length

[10, 10, 10, 10]

In [10]:
import pandas as pd

In [11]:
df = pd.DataFrame(list(zip(name_list, stars_list, num_of_reviews_list, address_list)),
                 columns=['Restaurant', 'Stars', 'Number Of Reviews', 'Address'], index=None)

In [12]:
df

,Restaurant,Stars,Number Of Reviews,Address
0,Leaf,4.5,51,
1,George’s Great British Kitchen,4.5,3,
2,The Smugglers Cove,4.0,31,
3,Docklands Fish and Chips,4.5,8,
4,Famous Blue Star Fish & Chip Shop,5.0,3,
...,...,...,...,...
235,Wahaca Liverpool,4.0,9,
236,Tribeca,4.0,13,
237,Eagle & Child,5.0,1,
238,The Old Riding School Cafe,2.0,1,


In [13]:
df.to_csv('liverpool.csv')